### **DLT Pipeline**

In [0]:
import dlt
from pyspark.sql.functions import *

In [0]:
#expectations
products_rules = {"valid product_id" : "product_id IS NOT NULL",
                  "valid product_name" : "product_name IS NOT NULL",
                  "valid product_price" : "price > 0"}

In [0]:
# create streaming table
@dlt.table()
@dlt.expect_all_or_drop(products_rules)
def dim_products_stage():
    df = spark.readStream.table("databricks_dev.silver.silver_products")\
        .withColumn("ingest_date", current_timestamp())
    return df

In [0]:
# create streaming view
@dlt.view()
def dim_products_view():
    df = spark.readStream.table("LIVE.dim_products_stage")
    return df

In [0]:
# create dim_products streaming table
dlt.create_streaming_table("dim_products")

In [0]:
dlt.apply_changes(
    target = "dim_products",
    source = "dim_products_view",
    keys = ["product_id"],
    sequence_by = "ingest_date",
    stored_as_scd_type = 2
)